In [70]:
import pandas as pd
import numpy as np
import ast
from glob import glob

In [39]:
def s_to_dict(s):
    try:
        return ast.literal_eval(s)
    except:
        return


In [103]:
def is_initial(name):
    return len(name) < 4 and '.' in name
    

In [3]:
DATA = './data/'

In [77]:
authors_df = pd.DataFrame()
for f in glob(DATA+'papers*csv'):
    df = pd.read_csv(f)
    df = df.iloc[:,5:6]
    df['authors'] = df.authors.apply(lambda x: x.split('||')) #spliteo los jsons de autores en listas
    df['authors'] = df.authors.apply(lambda x: [ s_to_dict(i) for i in x ]) # convierto los str json en dictsauthors.
    lst_col = 'authors'
    df = pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_col].str.len())
            for col in df.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns.tolist()]
    
    df = df.dropna()
    df['first_name'] = df.authors.apply(lambda x: x['first_name'])
    df['author_id'] = df.authors.apply(lambda x: x['id'])
    df['full_name'] = df.authors.apply(lambda x: x['name'])
    df = df[['first_name', 'author_id', 'full_name']]
    authors_df = authors_df.append(df)
    

In [81]:
len(authors_df)
len(authors_df.drop_duplicates())

53509

In [83]:
import gender_guesser.detector as gender
d = gender.Detector()

In [89]:
d.get_gender("Bob")

'male'

In [85]:
authors_df.head()

,first_name,author_id,full_name
0,Richard,1717841,Richard Szeliski
1,Arthur,48898002,Arthur D. Edelstein
2,Nenad,8610628,Nenad Amodaj
3,Karl,16207529,Karl Hoover
4,Ron,145127145,Ron D. Vale


In [86]:
author_names = authors_df[['first_name']]
author_names.head()

,first_name
0,Richard
1,Arthur
2,Nenad
3,Karl
4,Ron


In [90]:
author_names = author_names.drop_duplicates()
author_names['gender'] = author_names.first_name.apply(lambda x: d.get_gender(x))

In [110]:
author_names['is_initial'] = author_names.first_name.apply(is_initial)
author_names.is_initial.value_counts()


False    16966
True        39
Name: is_initial, dtype: int64

In [118]:
author_names = author_names[~ author_names.is_initial]

In [120]:
author_names.to_csv(DATA+'first_name_gender.csv', index=False)

In [93]:
len(author_names)

17005

In [92]:
author_names.gender.value_counts()

unknown          10101
male              3842
female            1941
andy               741
mostly_male        249
mostly_female      131
Name: gender, dtype: int64

In [111]:
authors_df.first_name.apply(is_initial).value_counts()

False    66440
True      1665
Name: first_name, dtype: int64

In [97]:
len(authors_df)

68105

In [96]:
authors_df.merge(author_names, on='first_name').gender.value_counts()

male             35032
unknown          18724
female            6631
andy              5413
mostly_male       1758
mostly_female      547
Name: gender, dtype: int64

In [100]:
author_names[author_names.gender == 'andy']

,first_name,gender
53,Yi,andy
77,Bin,andy
90,Chun-Liang,andy
135,Jian,andy
136,Jie,andy
160,An,andy
161,Po,andy
175,Xiao,andy
176,Hao,andy
254,Cheng,andy


In [121]:
authors_df['is_initial'] = authors_df.first_name.apply(is_initial)
authors_df['gender'] = authors_df.first_name.apply(lambda x: d.get_gender(x))

In [123]:
authors_df.to_csv(DATA+'author_id_name_gender.csv')